In [49]:
import os
# Provides a way of using operating system-dependent functionality like reading or writing to the file system.

from langchain_community.document_loaders import PyPDFLoader
# PyPDFLoader is used for loading and processing PDF documents, typically to extract text from them.

from dotenv import load_dotenv
# load_dotenv is used to load environment variables from a .env file into the Python environment.

from langchain_community.llms import Ollama 
# Ollama is a specific implementation of a large language model (LLM) available in the langchain_community package.

from langchain_core.output_parsers import StrOutputParser
# StrOutputParser is used to parse and process the output of language models into strings or other formats.

from langchain.prompts import PromptTemplate
# PromptTemplate helps in creating and managing prompt templates for interacting with language models.

from langchain_community.embeddings import OllamaEmbeddings
# OllamaEmbeddings provides functionality to generate embeddings (numerical representations) using the Ollama model.

from langchain_community.vectorstores import DocArrayInMemorySearch
# DocArrayInMemorySearch is a vector store implementation for performing similarity searches on embeddings in memory.


load_dotenv()

MODEL = "llama2" # Model used - 3.8 GB
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()


parser = StrOutputParser() # Removes "AI Response"

loader = PyPDFLoader("Quantum Computing Principles.pdf") # Loads PDF
pages = loader.load_and_split() #Splits PDF by pages
pages # Stores pages in local memory


[Document(metadata={'source': 'Quantum Computing Principles.pdf', 'page': 0}, page_content='Journal of International T echnology and Information Management Journal of International T echnology and Information Management \nVolume 29 Issue 2 Article 3 \n2020 \nQuantum Computing: Principles and Applications Quantum Computing: Principles and Applications \nYoshit o Kanamori \nUniv ersity of Alaska Anchor age, ykanamori@alaska.edu \nSeong-Moo Y oo \nUniv ersity of Alabama in Huntsville , yoos@uah.edu \nFollow this and additional works at: https:/ /scholar works.lib.csusb.edu/jitim \n Part of the Communication T echnology and New Media Commons , Computer and Systems \nArchitectur e Commons , Information Security Commons , Management Information Systems Commons , \nScience and T echnology Studies Commons , Technology and Inno vation Commons , and the Theor y and \nAlgorithms Commons \nRecommended Citation Recommended Citation \nKanamori, Y oshit o and Y oo, Seong-Moo (2020) "Quantum Computing

In [50]:
# ----------TEST---------- #
template = """
Answer the question based on the context below. If you cant
answer the question, reply "I dont know"

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you cant
answer the question, reply "I dont know"

Context: Here is some context

Question: Here is a question




In [51]:
chain = prompt | model | parser # creates a langchain : feed it a prompt ---> model ---> parser


In [52]:
chain.input_schema.schema() # Tells you what parameters ask for

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question']}

In [56]:
# ----------TEST---------- #
print(chain.invoke(
    {
        "context": "Lebron is the greatest basketball player of all time.",
        "question": "Who's the greatest basketball player?"
    }
))

Of course! Based on the context you provided, the answer to the question is clear: Lebron James is the greatest basketball player of all time.


In [57]:
# Storing Vectors
vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
    )

In [58]:
retriever = vectorstore.as_retriever()
retriever.invoke("Quantum Computing Principles.pdf")

[Document(metadata={'source': 'Quantum Computing Principles.pdf', 'page': 12}, page_content='Quantum Computing: Principles and Applications   Kanamori  & Yoo  \n   \n©International Information M anagement Association, Inc. 2020  54         ISSN: 1941 -6679 -On-line Copy  \n.  \nFigure 6: Initial States for N = 8 (n = 3) \n \nNext, a quantum circuit is used to flip the phase of the state where x is equal to 𝑥𝑎 \n(Figure 7). \n|ψ′⟩=(−1)f(x)∑ρx|x⟩N−1\nx=0 \n=ρ0|0⟩+ρ1|1⟩+⋯+(−1)ρxa|xa⟩+⋯+ρN−1|N−1⟩(16) \n \nwhere 𝑓(𝑥)=[0𝑥≠𝑥𝑎\n1𝑥=𝑥𝑎 . \n \n \n \nFigure 7: Flip the phase of the state xa for N = 8 (n = 3), assuming \n𝒇(𝒙𝒂=𝟑)=𝟏 in this example  \nThen, the difference between the average of the probability amplitudes 𝜌𝑎𝑣𝑒 and the \namplitude 𝜌𝑥 is subtracted from the amplitude 𝜌𝑎𝑣𝑒 (Figure 8)  \n \nwhere 𝜌𝑎𝑣𝑒=1\n𝑁∑𝜌𝑥𝑁−1\n𝑥=0. \n|𝜓′′⟩=∑{𝜌𝑎𝑣𝑒−(𝜌𝑥−𝜌𝑎𝑣𝑒)}|𝑥⟩=∑(2𝜌𝑎𝑣𝑒−𝜌𝑥)|𝑥⟩   𝑁−1\n𝑥=0𝑁−1\n𝑥=0(17)'),
 Document(metadata={'source': 'Quantum Computing Principles.pdf', 'page': 10}, page_content='Quantum Co

In [60]:
from operator import itemgetter
# Constructs callable objects that extract items from iterable objects like lists or dictionaries based on their indices or keys

# Query
chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question")
} | prompt | model | parser

# Change prompt below to ask a different question
print(chain.invoke({"question": "What is Quantum Computing?"}))



Quantum computing is a branch of quantum mechanics that explores the potential of quantum-based information processing and computation. It is a rapidly developing field that seeks to leverage the unique properties of quantum systems, such as superposition, entanglement, and interference, to perform calculations and operations that are beyond the capabilities of classical computers.

In quantum computing, qubits (quantum bits) are used instead of classical bits to represent and process information. Qubits can exist in multiple states simultaneously, allowing them to process multiple inputs simultaneously as well. This property enables quantum computers to solve certain problems much faster than classical computers, particularly those that involve complex calculations or large amounts of data.

Quantum computing has a wide range of potential applications, including:

1. Cryptography: Quantum computers can break some classical cryptography systems, but they can also be used to create new,